In [81]:
import pandas as pd
import numpy as np


In [82]:
loandata = pd.read_csv('data/lending-club-data.csv', dtype={'url':'string', 'next_pymnt_d':'string'})

In [83]:
loandata['safe_loan'] = loandata['bad_loans'].apply(lambda x: +1 if x==0 else -1)
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loan'
loandata = loandata[features + [target]]
#loandata = loandata[features]
print loandata.head()
print loandata.columns.values

  grade        term home_ownership emp_length  safe_loan
0     B   36 months           RENT  10+ years          1
1     C   60 months           RENT   < 1 year         -1
2     C   36 months           RENT  10+ years          1
3     C   36 months           RENT  10+ years          1
4     A   36 months           RENT    3 years          1
['grade' 'term' 'home_ownership' 'emp_length' 'safe_loan']


In [84]:
# one hot encoding converts the data aset into machine splittable version so we are not deailing
# with strings
loans_ohe = pd.get_dummies(loandata)
loans_ohe.head()
ohe_features= loans_ohe.columns.values
type(ohe_features)
print ohe_features

['safe_loan' 'grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F'
 'grade_G' 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']


In [120]:
import json

with open('data/module-5-assignment-2-train-idx.json', 'r') as f: 
    train_idx = json.load(f)
f.close()

with open('data/module-6-assignment-validation-idx.json','r') as f:
    validation_idx = json.load(f)
f.close()

In [121]:
# select the specific samples from the training and validation set in the loans one hot encoded data
train_data = loans_ohe.iloc[train_idx]
validation_data= loans_ohe.iloc[validation_idx]

In [122]:
#safe_loans_raw = loandata[loandata[target] == 1]
#risky_loans_raw = loandata[loandata[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
#percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
#safe_loans = safe_loans_raw.sample(frac=percentage, random_state = 1)
#risky_loans = risky_loans_raw
#loans_data = risky_loans.append(safe_loans)

#print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
#print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
#print "Total number of loans in our new dataset :", len(loans_data)

In [123]:
len(loans_ohe.columns.values)

26

In [124]:
#train_data = loans_ohe.sample(frac=.8,random_state=1)
#test_data = loans_ohe.drop(train_data.index)

In [125]:
len(train_data)

37224

In [126]:
len(test_data)

9284

In [127]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    ## YOUR CODE HERE
    return True if len(data) <= min_node_size else False

** quiz question: we should stop since the data size is smaller than the minimum required to make a decision**


In [128]:
def error_reduction(error_before_split, error_after_split):
    #return the error before the split minus the error after the split
    return error_before_split - error_after_split

** The error of the node is 3/(3+6) = 1/3. (error_before_split). all 4 error reductions are < .2, so it should stop**

In [129]:
def intermediate_node_num_mistakes(labels_in_node):
    #corner caseL if labes_in_node is empty, return 0
    #print type(labels_in_node)
    #print len(labels_in_node)
    #print labels_in_node[0]
    if len(labels_in_node) == 0:
        return 0
    # count the number of 1's (safe loans)
    safe_loans = [x for x in labels_in_node if x == 1]

    # count unsafe loans
    #unsafe_loans = filter(lambda x: x == -1, labels_in_node)
    unsafe_loans = [x for x in labels_in_node if x == -1]

    #print "safe loans = %d unsafe loans = %d" % (len(safe_loans), len(unsafe_loans))
    return len(safe_loans) if  len(safe_loans) < (len(unsafe_loans)) else len(unsafe_loans)

In [130]:
def best_splitting_feature(data,features,target):
    target_values = data[target]
    best_feature = None
    best_error = 10
    
    #convert to float
    num_data_points = float(len(data))
    mfeatures = np.delete(features,0)
    #consider each feature to decide what to split on
    for feature in mfeatures:
        
        left_split = data[data[feature] == 0]
        #print left_split[target]
        
        right_split = data[data[feature] == 1]
        
        #left side misclassicifed
        
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        
        error = (left_mistakes + right_mistakes)/float(num_data_points)
        #print "error %f feature %s" % (error, feature)
        if error < best_error:
            #print "setting best feature to %s for error %f" %(feature, error)
            best_error = error
            best_feature = feature
        
    print "Best Feature is %s" % best_feature        
    return best_feature
        

In [131]:
def create_leaf(target_values):
    #a leaf node
    leaf = {'splitting_feature': None,
            'left': None,
            'right': None,
            'is_leaf': True
           }
    # count the number of data points that are +1 and -1 in this node
    
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # for the leaf node, set the prediction to be the majority class
    # store the prediced class (1 or -1) in leaf['prediction']
    
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1
    else:
        leaf['prediction'] = -1
        
    return leaf
    

In [172]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10, min_error_reduction=0.0, min_node_size=1):
    
    target_values = data[target]
    #data.drop(target,axis=1)
    remaining_features = features[:] # Make a copy of the features.

    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if  len(target_values)== 0:  ## YOUR CODE HERE
        print "Stopping condition 1 reached."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0 :   ## YOUR CODE HERE
        print "Stopping condition 2 reached."    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth :  ## YOUR CODE HERE
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    
    
    print "data len is %d" %len(data)
    if reached_minimum_node_size(data,min_node_size):
        print "++++++++++Reached minimum node size - returning leaf"
        return create_leaf(target_values)
    
    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data,features,target)
    print "**** Splitting on %s" % splitting_feature
    # Split on the best feature that we found. 
    left_split  = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] ==1]
    
    
    error_before_split = intermediate_node_num_mistakes(target_values)/float(len(data))
      
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes =    intermediate_node_num_mistakes(left_split[target])
    right_mistakes =   intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    print "before mistakes %d, after mistakes %d %d" % (intermediate_node_num_mistakes(target_values),left_mistakes, right_mistakes)
    print "=======error before, error after %f %f" %(error_before_split, error_after_split)
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print "+++++++Early stopping condition 3 reached. Minimum error reduction. %f" %(error_after_split)
        return   create_leaf(target_values)
    
    #print splitting_feature
    #print remaining_features
    itemindex = np.where(remaining_features==splitting_feature)[0]
    remaining_features = np.delete(remaining_features,itemindex[0])
    print "***"
    print "splitting feature %s at index %d should be removed" % (splitting_feature, itemindex[0])
    print remaining_features
    print "***"

    #remaining_features.drop(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])
        ## YOUR CODE HERE

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth, min_error_reduction, min_node_size)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features,target, current_depth+1, max_depth, min_error_reduction, min_node_size)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [173]:
my_decision_tree_new = decision_tree_create(train_data,ohe_features,target,max_depth=6, min_node_size = 100, min_error_reduction=0.0)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data len is 37224
Best Feature is term_ 36 months
**** Splitting on term_ 36 months
before mistakes 18476, after mistakes 3221 12474
=======error before, error after 0.496346 0.421637
***
splitting feature term_ 36 months at index 8 should be removed
['safe_loan' 'grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F'
 'grade_G' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
***
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data len is 9223
Best Feature 

In [174]:
my_decision_tree_old = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data len is 37224
Best Feature is term_ 36 months
**** Splitting on term_ 36 months
before mistakes 18476, after mistakes 3221 12474
=======error before, error after 0.496346 0.421637
***
splitting feature term_ 36 months at index 8 should be removed
['safe_loan' 'grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F'
 'grade_G' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
***
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data len is 9223
Best Feature 

In [152]:
def classify(tree,x, annotate = False):
    # if the node is a leaf node
    if tree['is_leaf']:
        if annotate:
            print "at leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:
        # split on feature
        split_feature_value =x[tree['splitting_feature']]
        if annotate:
            print "Split on %s=%s" %(tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'],x,annotate)
        else:
            return classify(tree['right'], x, annotate)

In [153]:
print my_decision_tree_old

term_ 36 months


In [159]:
print validation_data.iloc[0][my_decision_tree_old['splitting_feature']]
print my_decision_tree_new

0
{'is_leaf': False, 'splitting_feature': 'term_ 36 months', 'right': {'is_leaf': False, 'splitting_feature': 'grade_D', 'right': {'is_leaf': True, 'splitting_feature': None, 'right': None, 'prediction': -1, 'left': None}, 'prediction': None, 'left': {'is_leaf': False, 'splitting_feature': 'grade_E', 'right': {'is_leaf': True, 'splitting_feature': None, 'right': None, 'prediction': -1, 'left': None}, 'prediction': None, 'left': {'is_leaf': False, 'splitting_feature': 'grade_F', 'right': {'is_leaf': False, 'splitting_feature': 'emp_length_8 years', 'right': {'is_leaf': True, 'splitting_feature': None, 'right': None, 'prediction': 1, 'left': None}, 'prediction': None, 'left': {'is_leaf': True, 'splitting_feature': None, 'right': None, 'prediction': -1, 'left': None}}, 'prediction': None, 'left': {'is_leaf': False, 'splitting_feature': 'emp_length_n/a', 'right': {'is_leaf': False, 'splitting_feature': 'grade_A', 'right': {'is_leaf': True, 'splitting_feature': None, 'right': None, 'predict

In [157]:
print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0], annotate=True)

Split on term_ 36 months=0
Split on grade_A=0
at leaf, predicting -1
Predicted class: -1 


In [158]:
print 'Predicted class: %s ' % classify(my_decision_tree_old, validation_data.iloc[0], annotate=True)

Split on term_ 36 months=0
Split on grade_A=0
Split on grade_B=0
Split on grade_C=0
Split on grade_D=1
at leaf, predicting -1
Predicted class: -1 


** decision_tree_new is shorter**

** new decision tree is always shorter because of stopping conditions**

** max is 6 because of max_depth**

In [162]:
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} | {}'.format(*x)

def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    difference = np.where(prediction != data['safe_loan'])
    print len(difference[0])
    print float(len(difference[0]))/float(len(data['safe_loan']))

In [163]:
evaluate_classification_error(my_decision_tree_old, validation_data)

3563
0.383778543731


In [164]:
evaluate_classification_error(my_decision_tree_new, validation_data)

3562
0.383670831538


** Validation error of new tree is lower**

Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (too small, just right, and too large).

22. Train three models with these parameters:

model_1: max_depth = 2 (too small)
model_2: max_depth = 6 (just right)
model_3: max_depth = 14 (may be too large)
For each of these three, set min_node_size = 0 and min_error_reduction = -1. Make sure to call the models model_1, model_2, and model_3.

Note: Each tree can take up to a few minutes to train. In particular, model_3 will probably take the longest to train.

23. Let us evaluate the models on the train and validation data. Let us start by evaluating the classification error on the training data. Your code should be analogous to:





In [166]:
model_1 = decision_tree_create(train_data, ohe_features, target, max_depth =2, min_node_size = 0, min_error_reduction=-1)
model_2 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 0, min_error_reduction=-1)
model_3 = decision_tree_create(train_data, ohe_features, target, max_depth =14, min_node_size = 0, min_error_reduction=-1)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data len is 37224
Best Feature is term_ 36 months
**** Splitting on term_ 36 months
before mistakes 18476, after mistakes 3221 12474
=======error before, error after 0.496346 0.421637
***
splitting feature term_ 36 months at index 8 should be removed
['safe_loan' 'grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F'
 'grade_G' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
***
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data len is 9223
Best Feature 

In [168]:
evaluate_classification_error(model_1, validation_data)

3696
0.398104265403


In [170]:
evaluate_classification_error(model_2, validation_data)

3563
0.383778543731


In [171]:
evaluate_classification_error(model_3, validation_data)

3503
0.37731581215


3. Let us evaluate the models on the train and validation data. Let us start by evaluating the classification error on the training data. Your code should be analogous to:



In [175]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data)

Training data, classification error (model 1): 14891
0.400037610144
None
Training data, classification error (model 2): 14214
0.381850419084
None
Training data, classification error (model 3): 14003
0.376182033097
None


In [176]:
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, validation_data)
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, validation_data)
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, validation_data)

Training data, classification error (model 1): 3696
0.398104265403
None
Training data, classification error (model 2): 3563
0.383778543731
None
Training data, classification error (model 3): 3503
0.37731581215
None


Quiz Question: Which tree has the smallest error on the validation data? 
   
   ** Model 3 **

Quiz Question: Does the tree with the smallest error in the training data also have the smallest error in the validation data?

**Yes**

Quiz Question: Is it always true that the tree with the lowest classification error on the training set will result in the lowest classification error in the validation set?

**No**

In [177]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [178]:
count_leaves(model_1)

4

In [179]:
count_leaves(model_2)

19

In [180]:
count_leaves(model_3)

41

Quiz question: Which tree has the largest complexity?
    
    **Model_3**

Quiz question: Is it always true that the most complex tree will result in the lowest classification error in the validation_set?
    
    **No**

In [189]:
model_4 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 0, min_error_reduction=-1)
model_5 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 0, min_error_reduction=0)
model_6 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 0, min_error_reduction=5)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data len is 37224
Best Feature is term_ 36 months
**** Splitting on term_ 36 months
before mistakes 18476, after mistakes 3221 12474
=======error before, error after 0.496346 0.421637
***
splitting feature term_ 36 months at index 8 should be removed
['safe_loan' 'grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F'
 'grade_G' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
***
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data len is 9223
Best Feature 

In [184]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data)

 Validation data, classification error (model 4): 3563
0.383778543731
None
Validation data, classification error (model 5): 3563
0.383778543731
None
Validation data, classification error (model 6): 4674
0.503446790177
None


In [186]:
print count_leaves(model_4)
print count_leaves(model_5)
print count_leaves(model_6)

19
13
1


Quiz Question: Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?

** 4 has largest complexity - yes, since we have disabled error reduction **

Quiz Question: model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?

    No. Model 6 converges too quickly

In [190]:
model_7 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 0, min_error_reduction=-1)
model_8 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 2000, min_error_reduction=-1)
model_9 = decision_tree_create(train_data, ohe_features, target, max_depth =6, min_node_size = 50000, min_error_reduction=-1)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data len is 37224
Best Feature is term_ 36 months
**** Splitting on term_ 36 months
before mistakes 18476, after mistakes 3221 12474
=======error before, error after 0.496346 0.421637
***
splitting feature term_ 36 months at index 8 should be removed
['safe_loan' 'grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F'
 'grade_G' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
***
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data len is 9223
Best Feature 

In [191]:
print "Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data)
print "Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data)
print "Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data)

Validation data, classification error (model 7): 3563
0.383778543731
None
Validation data, classification error (model 8): 3570
0.384532529082
None
Validation data, classification error (model 9): 4674
0.503446790177
None


In [192]:
print count_leaves(model_7)
print count_leaves(model_8)
print count_leaves(model_9)

19
12
1


**Model 7 is most reasonable**